# Crashes

In [ ]:
crashes_only = cleaned_crashes[['CollisionID','CrashDate','NumberOfInjured']].copy()

# Adds WeatherID to Crashes
#
# crashes_only['CrashDate'] = pd.to_datetime(crashes_only['CrashDate'], format='%m/%d/%Y')
# epoch = datetime(1970, 1, 1)
# for index, row in crashes_only.iterrows():
# 	delta = (row['CrashDate'] - epoch)
# 	crashes_only.loc[index, 'WeatherID'] = delta.days
# crashes_only['WeatherID'] = crashes_only['WeatherID'].astype(int)

crashes_only.drop(columns=['CrashDate'], inplace=True)
print(crashes_only)

crashes_only.to_csv('Crashes.csv', index=False)

## Location

Planänderung: Datenbankentwurf wie im Diagramm dargestellt, mit hinzugefügten Fremdschlüsseln in `Neigbourhoods` und `ZipCodes`. Die `ZipCodeID` entfällt, da der `ZipCode` bereits eindeutig ist. Die Fremdschlüssel `BoroughID` und `NeigbourhoodID` entfallen demach in der `Location` Tabelle. Die `ZipCodeID` entfällt/wird direkt durch den `ZipCode` ersetzt. Des weiteren wird die `LocationID` durch die `CollisionID` ersetzt (zwischen `Location` und `Crashes` besteht eine 1:1 Beziehung).

In [ ]:
location = cleaned_crashes[['CollisionID','ZipCode','Latitude','Longitude']].copy()
location[['ZipCode']] = location[['ZipCode']].fillna(0)
location[['ZipCode']] = location[['ZipCode']].replace('', 0)
print(location)
location[['ZipCode']] = location[['ZipCode']].round(0)
location.to_csv('Location.csv', index=False)

## Time

Time hat aktuell ebenfalls `CollisionID` statt `TimeID`.

In [ ]:
time = cleaned_crashes[['CollisionID','CrashDate','CrashTime']].copy()
time['CrashDateTime'] = pd.to_datetime(time['CrashDate'] + time['CrashTime'], format='%m/%d/%Y%H:%M')
print("Number of unique days: " + str(time['CrashDate'].nunique()))
print("Earliest date: " + str(time["CrashDateTime"].min()))
print("Latest date: " + str(time["CrashDateTime"].max()))

# Achtung, Berechnung der WeatherID dauert mehrere Minuten (da for-each)
time['CrashDate'] = pd.to_datetime(time['CrashDate'], format='%m/%d/%Y')
epoch = datetime(1970, 1, 1)
for index, row in crashes_only.iterrows():
	delta = (row['CrashDate'] - epoch)
	time.loc[index, 'WeatherID'] = delta.days
time['WeatherID'] = time['WeatherID'].astype(int)


time.drop(columns=['CrashDate', 'CrashTime'], inplace=True)
print(time)

time.to_csv('Time.csv', index=False)

In [ ]:
import pandas as pd
from datetime import datetime

crashes = pd.read_csv('mvc_crashes.csv')
print(crashes.columns.values)

cleaned_crashes = crashes[['COLLISION_ID','CRASH DATE','CRASH TIME','BOROUGH','ZIP CODE','LATITUDE','LONGITUDE','NUMBER OF PERSONS INJURED']].copy()
cleaned_crashes.rename(columns={"COLLISION_ID": "CollisionID",
								"CRASH DATE": "CrashDate",
								"CRASH TIME": "CrashTime",
								"BOROUGH": "Borough",
								"ZIP CODE": "ZipCode",
								"LATITUDE": "Latitude",
								"LONGITUDE": "Longitude",
								"NUMBER OF PERSONS INJURED": "NumberOfInjured"
								}, inplace=True)

## Weather

Es gab zwischen dem 01.07.2012 und dem 08.09.2023 an 4087 Tagen Unfälle (sind wahrscheinlich alle Tage in dem Zeitraum). Wetterinformationen sind aktuell einmal pro Tag vorhanden (nicht stündlich).

`WeatherID` wird berechnet über die Anzahl der Tage seit dem 01.01.1970.

Inhalt der CSV:
- mittlere Tagestemperatur in °C
- Sunrise/Sunset nach ISO 8601
- Summe der Regenmenge in mm
- Summe der Schneehöhe in cm
- maximale Windgeschwindigkeit in km/h


In [ ]:
from datetime import datetime

weather = pd.read_csv("open-meteo-edit.csv")
weather['time'] = pd.to_datetime(weather['time'])
epoch = datetime(1970, 1, 1)
for index, row in weather.iterrows():
	delta = (row['time'] - epoch)
	weather.loc[index, 'WeatherID'] = delta.days
weather['WeatherID'] = weather['WeatherID'].astype(int)

weather.drop(columns=['time'], inplace=True)
weather.rename(columns={"temperature_mean": "Temperature",
						"sunrise": "Sunrise",
						"sunset": "Sunset",
						"rain_sum": "Rainfall",
						"snowfall_sum": "Snowfall",
						"windspeed_max": "Windspeed"
						}, inplace=True)
print(weather)
weather.to_csv("Weather.csv", index=False)

# Vehicles

In [ ]:
import pandas as pd
vehicles = pd.read_csv('mvc_vehicles.csv')

In [ ]:
cleaned_vehicles = vehicles[['UNIQUE_ID','COLLISION_ID','VEHICLE_ID','VEHICLE_TYPE','VEHICLE_YEAR','VEHICLE_MAKE','VEHICLE_MODEL']].copy()
cleaned_vehicles.rename(columns={"UNIQUE_ID": "UniqueID",
								"COLLISION_ID": "CollisionID",
								"VEHICLE_ID": "VehicleID",
								"VEHICLE_TYPE": "VehicleType",
								"VEHICLE_YEAR": "VehicleYear",
								"VEHICLE_MAKE": "VehicleMake",
								"VEHICLE_MODEL": "VehicleModel"
								}, inplace=True)
cleaned_vehicles.to_csv('cleaned_vehicles.csv', index=False)

# Persons

In [ ]:
import pandas as pd
persons = pd.read_csv('mvc_persons.csv')

In [ ]:
cleaned_persons = persons[['UNIQUE_ID', 'PERSON_ID','COLLISION_ID','VEHICLE_ID','PERSON_AGE','PERSON_INJURY','EMOTIONAL_STATUS','PERSON_SEX','PED_ROLE']].copy()
cleaned_persons.rename(columns={"UNIQUE_ID": "UniqueID",
								"PERSON_ID": "PersonID",
								"COLLISION_ID": "CollisionID",
								"VEHICLE_ID": "VehicleID",
								"PERSON_AGE": "Age",
								"PERSON_INJURY": "Injury",
								"EMOTIONAL_STATUS": "EmotionalStatus",
								"PERSON_SEX": "Gender",
								"PED_ROLE": "PedRole"
								}, inplace=True)
cleaned_persons.to_csv('cleaned_person.csv', index=False)